In [ ]:
!nvidia-smi

Wed Jun  7 17:43:20 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   77C    P0    32W /  70W |   2653MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
!zip -r /content/gdrive/MyDrive/test_videos.zip /content/gdrive/MyDrive/test_videos

In [ ]:
!nvcc -V

!gcc --version

!which python

%pip install torch==1.10.0+cu111 torchvision==0.11.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html

%pip install -U openmim
!mim install mmengine
!mim install "mmcv>=2.0.0rc1"
!mim install "mmdet>=3.0.0rc0"

!git clone https://github.com/open-mmlab/mmpose.git

%cd mmpose

%pip install -v -e .

!pip install -r requirements.txt

In [ ]:
# Check Pytorch installation
import torch, torchvision

print('torch version:', torch.__version__, torch.cuda.is_available())
print('torchvision version:', torchvision.__version__)

# Check MMPose installation
import mmpose

print('mmpose version:', mmpose.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version

print('cuda version:', get_compiling_cuda_version())
print('compiler information:', get_compiler_version())

torch version: 2.0.1+cu118 True
torchvision version: 0.15.2+cu118
mmpose version: 1.0.0
cuda version: 11.8
compiler information: GCC 9.3


In [ ]:
!pip install --upgrade pip
!pip install --upgrade numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 86.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.24.3 which is incompatible.
tensorflow 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.24.3 which is incompatible.


In [ ]:
!wget https://prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com/7w7fccy7ky-4.zip

--2023-06-06 15:15:36--  https://prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com/7w7fccy7ky-4.zip
Resolving prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com (prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com)... 52.218.89.152, 3.5.65.1, 52.92.20.210, ...
Connecting to prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com (prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com)|52.218.89.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8326349365 (7.8G) [application/zip]
Saving to: ‘7w7fccy7ky-4.zip’

7w7fccy7ky-4.zip    100%[===================>]   7.75G  47.0MB/s    in 2m 42s  

2023-06-06 15:18:18 (49.2 MB/s) - ‘7w7fccy7ky-4.zip’ saved [8326349365/8326349365]



In [ ]:
!unzip 7w7fccy7ky-4.zip

In [ ]:
%cd /content/mmpose/

import mmcv
from mmcv import imread
import mmengine
from mmengine.registry import init_default_scope
import numpy as np

from mmpose.apis import inference_topdown
from mmpose.apis import init_model as init_pose_estimator
from mmpose.evaluation.functional import nms
from mmpose.registry import VISUALIZERS
from mmpose.structures import merge_data_samples

try:
    from mmdet.apis import inference_detector, init_detector
    has_mmdet = True
except (ImportError, ModuleNotFoundError):
    has_mmdet = False

local_runtime = False

try:
    from google.colab.patches import cv2_imshow  # Colab için resim görselleştirme
except:
    local_runtime = True

pose_config = 'configs/body_2d_keypoint/topdown_heatmap/coco/td-hm_hrnet-w48_udp-8xb32-210e_coco-384x288.py'
pose_checkpoint = 'https://download.openmmlab.com/mmpose/v1/body_2d_keypoint/topdown_heatmap/coco/td-hm_hrnet-w48_udp-8xb32-210e_coco-384x288-70d7ab01_20220913.pth'
det_config = 'demo/mmdetection_cfg/faster_rcnn_r50_fpn_coco.py'
det_checkpoint = 'https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth'

device = 'cuda:0'
cfg_options = dict(model=dict(test_cfg=dict(output_heatmaps=False)))


# build detector
detector = init_detector(
    det_config,
    det_checkpoint,
    device=device
)

# build pose estimator
pose_estimator = init_pose_estimator(
    pose_config,
    pose_checkpoint,
    device=device,
    cfg_options=cfg_options
)

# init visualizer
pose_estimator.cfg.visualizer.radius = 3
pose_estimator.cfg.visualizer.line_width = 1
visualizer = VISUALIZERS.build(pose_estimator.cfg.visualizer)

visualizer.set_dataset_meta(pose_estimator.dataset_meta)

/content/mmpose
Loads checkpoint by http backend from path: https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth


Downloading: "https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth" to /root/.cache/torch/hub/checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth


Loads checkpoint by http backend from path: https://download.openmmlab.com/mmpose/v1/body_2d_keypoint/topdown_heatmap/coco/td-hm_hrnet-w48_udp-8xb32-210e_coco-384x288-70d7ab01_20220913.pth


Downloading: "https://download.openmmlab.com/mmpose/v1/body_2d_keypoint/topdown_heatmap/coco/td-hm_hrnet-w48_udp-8xb32-210e_coco-384x288-70d7ab01_20220913.pth" to /root/.cache/torch/hub/checkpoints/td-hm_hrnet-w48_udp-8xb32-210e_coco-384x288-70d7ab01_20220913.pth


The model and loaded state dict do not match exactly

unexpected key in source state_dict: data_preprocessor.mean, data_preprocessor.std

06/07 15:32:00 - mmengine - WARNING - `Visualizer` backend is not initialized because save_dir is None.


In [ ]:

def visualize_img(img_path, detector, pose_estimator, visualizer,
                  show_interval, out_file):

    # predict bbox
    init_default_scope(detector.cfg.get('default_scope', 'mmdet'))
    detect_result = inference_detector(detector, img_path)
    pred_instance = detect_result.pred_instances.cpu().numpy()
    bboxes = np.concatenate(
        (pred_instance.bboxes, pred_instance.scores[:, None]), axis=1)
    bboxes = bboxes[np.logical_and(pred_instance.labels == 0,
                                   pred_instance.scores > 0.3)]
    bboxes = bboxes[nms(bboxes, 0.3)][:, :4]

    # predict keypoints
    pose_results = inference_topdown(pose_estimator, img_path, bboxes)
    data_samples = merge_data_samples(pose_results)

    # show the results
    img = mmcv.imread(img_path)
    img = mmcv.imconvert(img, 'bgr', 'rgb')

    image = np.zeros((480, 640, 3), dtype=np.uint8)
    image.fill(255)

    visualizer.add_datasample(
        'result',
        image,
        data_sample=data_samples,
        draw_gt=False,
        draw_heatmap=False,
        draw_bbox=False,
        show=False,
        wait_time=show_interval,
        out_file=out_file,
        kpt_thr=0.3)

    return pose_results

In [ ]:
img = '/content/cas400134.png'
visualize_img(
    img,
    detector,
    pose_estimator,
    visualizer,
    show_interval=0,
    out_file=None)

vis_result = visualizer.get_image();

cv2_imshow(vis_result[:,:,::-1]);

In [ ]:
%cd /content/

import os

folder_names = ['Fall backwards', 'Fall forward', 'Fall left', 'Fall right', 'Fall sitting', 'Hop', 'Kneel', 'Pick up object', 'Sit down', 'Walk']
parent_folder = 'dataset'

for i in range(1, 11):
  os.makedirs(f"dataset/Subject.{i}", exist_ok=True)
  subject_folder = f'Subject.{i}'
  for folder_name in folder_names:
    folder_path = os.path.join(parent_folder, subject_folder, folder_name)
    os.makedirs(folder_path, exist_ok=True)

/content


In [ ]:
%cd /content/
main_parent_folder = 'CAUCAFall'

for i in range(1, 11):
  subject_folder = f'Subject.{i}'
  subject_path = os.path.join(main_parent_folder, subject_folder)

  for folder_name in folder_names:
    folder_path = os.path.join(subject_path, folder_name)
    file_list = os.listdir(folder_path)
    png_list = [file_name for file_name in file_list if file_name.endswith('.png')]

    for png_file in png_list:
      png_path = os.path.join(folder_path, png_file)
      out_file = os.path.join('/content', parent_folder, subject_folder, folder_name, png_file)
      results = visualize_img(png_path, detector, pose_estimator, visualizer, show_interval=0, out_file=out_file)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
from torchvision import models, transforms, datasets
from PIL import Image

model = models.vgg16(pretrained=True)

print("Önceki Classifier Hali:")
print(model.classifier)

# Modelin sınıflandırma katmanlarını alın
classifier = list(model.classifier.children())

# Son iki tam bağlantılı katmanı kaldırın
classifier = classifier[:-3]

# Yeni sınıflandırma katmanını oluşturun
new_classifier = nn.Sequential(*classifier)

# Modelin sınıflandırma katmanını güncelleyin
model.classifier = new_classifier

# Son sınıflandırma katmanından sonra gelen parametreleri dondurucu yapın
for param in model.features.parameters():
    param.requires_grad = False

print("Sonraki Classifier Hali:")
print(model.classifier)

# Modeli GPU'ya taşıma
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:02<00:00, 275MB/s]


Önceki Classifier Hali:
Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)
Sonraki Classifier Hali:
Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
)


In [ ]:
model.load_state_dict(torch.load('/content/gdrive/MyDrive/vgg16_pose_model2.pth'))
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
img = "/content/cis700211.png"

In [ ]:
results= visualize_img(
    img,
    detector,
    pose_estimator,
    visualizer,
    show_interval=0,
    out_file=None)

vis_result = visualizer.get_image()

In [ ]:
image = Image.fromarray(vis_result)

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

input_image = transform(image).unsqueeze(0)  # Görüntüyü tensör formuna dönüştürme ve boyutunu genişletme


In [ ]:
import time
import cv2
import numpy as np
from PIL import Image
import torchvision.transforms as transforms

# Video dosyasını açın
video_path = "/content/gdrive/MyDrive/test_videos/subject8/WalkS8.avi"
cap = cv2.VideoCapture(video_path)

# Video özelliklerini alın
fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Video kaydedici
output_path = "/content/gdrive/MyDrive/test_videos/subject8/WalkS8_test.mp4"
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

# Font ayarları
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 1
font_thickness = 2
text_position = (int(frame_width/2), 30)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Frame'i pose tahmini ve görselleştirme için gönderme
    pose_results = visualize_img(frame, detector, pose_estimator, visualizer, show_interval=0, out_file=None)

    # Görselleştirme sonucunu alın
    vis_result = visualizer.get_image()

    # Görüntüyü PIL formatına dönüştürme
    pil_image = Image.fromarray(vis_result)

    # Görüntüyü dönüştürme
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    input_image = transform(pil_image).unsqueeze(0)

    # Görüntüyü CUDA cihazına gönderme
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    input_image = input_image.to(device)

    class_names = ['fall', 'non-fall']

    # Tahmin yapma
    output = model(input_image)
    _, predicted = torch.max(output, 1)
    print(class_names[predicted.item()])

    # Tahmin sonucunu görüntüye yazdırma
    predicted_class = class_names[predicted.item()]
    cv2.putText(frame, predicted_class, text_position, font, font_scale, (0, 255, 0), font_thickness)

    # Görüntüyü OpenCV formatına dönüştürme
    frame = cv2.cvtColor(np.array(frame), cv2.COLOR_RGB2BGR)

    # Video kaydediciye yazma
    out.write(frame)

out.release()
cap.release()


non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
non-fall
n

In [ ]:
# Görüntüyü CUDA cihazına gönderme (eğer kullanılıyorsa)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
input_image = input_image.to(device)

class_names = ['fall', 'non-fall']
# Tahmin yapma
output = model(input_image)
_, predicted = torch.max(output, 1)

print("Tahmin Sonucu:")
print(class_names[predicted.item()])


Tahmin Sonucu:
fall
